In [3]:
import os
os.chdir("../")

In [2]:
import pandas as pd

In [3]:
persona_scores_df = pd.read_csv("persona_scores.csv")
persona_scores_df.head()

,score,member,similarity_score,audience
0,0.000000,PARENT,1.000000,PARENT
1,0.922171,AUD0_0001,0.550822,Audience 0
2,0.937676,AUD0_0002,0.537676,Audience 0
3,0.955136,AUD0_0003,0.517461,Audience 0
4,0.955487,AUD0_0004,0.512164,Audience 0


In [4]:
persona_scores_df

,score,member,similarity_score,audience
0,0.000000,PARENT,1.000000,PARENT
1,0.922171,AUD0_0001,0.550822,Audience 0
2,0.937676,AUD0_0002,0.537676,Audience 0
3,0.955136,AUD0_0003,0.517461,Audience 0
4,0.955487,AUD0_0004,0.512164,Audience 0
5,0.907547,AUD0_0005,0.541504,Audience 0
6,0.938004,AUD1_0001,0.529243,Audience 1
7,1.023302,AUD1_0002,0.445300,Audience 1
8,0.953017,AUD1_0003,0.526153,Audience 1
9,0.960730,AUD1_0004,0.537944,Audience 1


In [5]:
df = persona_scores_df

# Optional: drop parent if you don't want it in the plot
df_plot = df[df["member"] != "PARENT"].copy()

In [6]:
import argparse
from pathlib import Path

import pandas as pd
import plotly.express as px


In [7]:
fig = px.scatter(
        df,
        x="score",
        y="similarity_score",
        color="audience",
        hover_data=["member"],
        title="Persona Distance vs Similarity to Parent",
    )

fig.update_layout(
    xaxis_title="Score (PCA distance from Parent)",
    yaxis_title="Similarity to Parent (cosine)",
    width=1000,
    height=700,
    legend_title="Audience",
)

# DISPLAY instead of writing HTML
fig.show()

In [9]:
%pip install "nbformat>=4.2.0"

  Using cached fastjsonschema-2.21.2-py3-none-any.whl.metadata (2.3 kB)
Using cached fastjsonschema-2.21.2-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [nbformat]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import plotly.express as px

# Load CSV (change filename if needed)
df = pd.read_csv("persona_scores_pca.csv")

# Create a separate 3D plot for each audience
for audience_label, df_aud in df.groupby("audience"):
    fig = px.scatter_3d(
        df_aud,
        x="pc1",
        y="pc2",
        z="pc3",
        color="similarity_score",  # or "score" / "audience" if you prefer
        hover_data=["member", "score", "similarity_score"],
        title=f"3D PCA Persona Space — {audience_label}",
    )

    fig.update_layout(
        scene=dict(
            xaxis_title="PC1",
            yaxis_title="PC2",
            zaxis_title="PC3",
        ),
        width=1000,
        height=800,
        legend_title="Similarity Score",
    )

    fig.show()


In [5]:
import pandas as pd
import plotly.express as px

# Load CSV
df = pd.read_csv("persona_scores_pca.csv")

# Loop per audience
for audience_label, df_aud in df.groupby("audience"):

    # Center the PCA coordinates for this audience
    df_aud = df_aud.copy()
    df_aud["pc1_c"] = df_aud["pc1"] - df_aud["pc1"].mean()
    df_aud["pc2_c"] = df_aud["pc2"] - df_aud["pc2"].mean()
    df_aud["pc3_c"] = df_aud["pc3"] - df_aud["pc3"].mean()

    # Plot centered PCA
    fig = px.scatter_3d(
        df_aud,
        x="pc1_c",
        y="pc2_c",
        z="pc3_c",
        color="similarity_score",  # or "score"
        hover_data=["member", "score", "similarity_score"],
        title=f"3D PCA (Centered) — {audience_label}",
    )

    fig.update_layout(
        scene=dict(
            xaxis_title="PC1 (centered)",
            yaxis_title="PC2 (centered)",
            zaxis_title="PC3 (centered)",
        ),
        width=1000,
        height=800,
        legend_title="Similarity",
    )

    fig.show()


In [9]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load CSV with columns: pc1, pc2, pc3, member, score, similarity_score, audience
df = pd.read_csv("persona_scores_pca.csv")

# One plot per audience
for audience_label, df_aud in df.groupby("audience"):

    df_aud = df_aud.copy()

    # ---- Center audience at (0,0,0) ----
    c1 = df_aud["pc1"].mean()
    c2 = df_aud["pc2"].mean()
    c3 = df_aud["pc3"].mean()

    df_aud["pc1_c"] = df_aud["pc1"] - c1
    df_aud["pc2_c"] = df_aud["pc2"] - c2
    df_aud["pc3_c"] = df_aud["pc3"] - c3

    # ---- Scatter plot for members ----
    fig = px.scatter_3d(
        df_aud,
        x="pc1_c",
        y="pc2_c",
        z="pc3_c",
        color="similarity_score",      # color by similarity (or "score" or "audience")
        hover_data=["member", "score", "similarity_score"],
        title=f"3D PCA — Audience Centered — {audience_label}",
    )

    # ---- Add the audience center as a different color/marker ----
    fig.add_trace(
        go.Scatter3d(
            x=[0], y=[0], z=[0],
            mode="markers+text",
            name="Audience Center",
            marker=dict(size=16, color="red", symbol="diamond"),
            text=["CENTER"],
            textposition="bottom center",
            hovertemplate="<b>Audience Center</b><extra></extra>",
        )
    )

    # ---- Layout ----
    fig.update_layout(
        scene=dict(
            xaxis_title="PC1 (centered)",
            yaxis_title="PC2 (centered)",
            zaxis_title="PC3 (centered)",
        ),
        width=1000,
        height=800,
        legend_title="Similarity",
    )

    fig.show()
